In [12]:
import pandas as pd

In [13]:
Data = pd.read_csv(r"C:\Users\Carlo\Desktop\Todo el proyecto\Data Posición\Delanteros.csv")
pd.set_option('display.max_columns', None)

In [14]:
import pandas as pd
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler
import plotly.graph_objects as go

Columna_habilidades = Data.columns.drop(["Name", "League"])

features = Data[Columna_habilidades]

# Crea un objeto StandardScaler
scaler = StandardScaler()

# Ajusta el escalador a tus datos y luego transforma tus datos
features_scaled = scaler.fit_transform(features)

# Inicializa el modelo NearestNeighbors
model = NearestNeighbors(n_neighbors=6)  # Buscamos 6 vecinos porque uno de ellos será el jugador mismo

# Ajusta el modelo a tus datos
model.fit(features_scaled)

caracteristicas_grafico = ['Goals', 'Big chances missed', 'Successful dribbles %', 'Total shots', 'Headed goals', 'Set piece conversion %', 'Assists']

def buscar_jugadores_similares(nombre, caracteristicas_grafico):
    jugador = Data[Data['Name'] == nombre]
    if len(jugador) == 0:
        return 'Jugador no encontrado'
    else:
        jugador_scaled = scaler.transform(jugador[features.columns])
        distancias, indices = model.kneighbors(jugador_scaled)
        jugadores_similares = Data.iloc[indices[0][1:]]  # Excluimos el primer resultado porque será el jugador mismo

        # Escala todas las características de los jugadores similares
        jugadores_similares_scaled = scaler.transform(jugadores_similares[features.columns])

        # Crea un gráfico de radar para cada jugador similar
        fig = go.Figure()
        for i, row in enumerate(jugadores_similares_scaled):
            fig.add_trace(go.Scatterpolar(
                r=row[features.columns.isin(caracteristicas_grafico)],
                theta=caracteristicas_grafico,
                fill='toself',
                name=jugadores_similares.iloc[i]['Name']
            ))
        fig.update_layout(
            autosize=False,
            width=800,
            height=800,
            polar=dict(
                radialaxis=dict(
                    visible=True,
                    range=[-2, 10]  # Ajusta este rango según tus datos
                )),
            showlegend=True
        )
        fig.show()

        return jugadores_similares

In [16]:
buscar_jugadores_similares('Rafael Leão', caracteristicas_grafico)

,Name,Goals,Big chances missed,Successful dribbles %,Total shots,Headed goals,Set piece conversion %,Interceptions,Big chances created,Assists,Total passes,Accurate passes %,Accurate final third passes,Accurate long balls %,Yellow cards,Red cards,Aerial duels won %,Total duels won %,Was fouled,Fouls,Appearances,Age Range,Position,League,Level
1282,Bryan Mbeumo,0.24,0.29,54.88,1.87,0.00,0.0,0.34,0.37,0.21,21.29,73.67,8.08,58.82,0.13,0.0,35.35,37.72,0.61,0.71,38,1,4,Inglaterra Premier League,1
2693,Lukas Daschner,0.26,0.29,40.91,2.03,0.00,0.0,0.47,0.29,0.18,23.74,78.81,7.24,72.22,0.12,0.0,33.33,35.29,0.74,0.76,34,2,4,Alemania Bundesliga 2,2
3311,Raphinha,0.19,0.22,46.39,2.19,0.08,0.0,0.39,0.47,0.19,28.97,79.87,10.36,64.58,0.22,0.0,60.87,49.80,1.03,0.61,36,2,4,España La Liga 1,1
1399,Ademola Lookman,0.42,0.23,47.89,1.68,0.03,0.0,0.13,0.32,0.19,19.29,78.60,7.71,42.86,0.10,0.0,18.75,39.81,1.16,0.77,31,2,4,Italia Serie A,1
3326,Iñaki Williams,0.28,0.39,49.47,2.36,0.06,0.0,0.19,0.28,0.08,19.72,71.13,6.31,53.85,0.06,0.0,45.71,43.56,0.86,0.86,36,2,4,España La Liga 1,1
